In [3]:
import difflib
import re
import string
import numpy as np 
import openai

In [4]:

#read the txt file contains OpenAI API key
with open('openai_api_key.txt') as f:
    api_key = f.readline()
openai.api_key = api_key


In [5]:
def proofread(text):
    system_role = "Korrigiere den folgenden deutschen Text und gib mir die korrigierte Version. Mach keine Erklärungen, gib nur den korrigierten Text zurück. Wenn der Text bereits korrekt ist, gib den Originaltext zurück"
    messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": text}
    ]

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    chat_response = completion.choices[0].message.content 
    return chat_response

def remove_commas_and_dots(text):
    return text.replace(',', '').replace('.', '')

def remove_space_before_punctuation(text):
    pattern = r'\s+([.?!,:;])'
    return re.sub(pattern, r'\1', text)

def count_words(s):
    return sum([i.strip(string.punctuation).isalpha() for i in s.split()])


In [6]:
def grammarly(text_wErrors):
    # print("\n\n\n\n\n\n\n")
    # print("GRAMMARLY")
    # print("#########")
    # print(f"text_wErrors {text_wErrors}")
    text_wErrors = remove_space_before_punctuation(text_wErrors)
    # print(f"text_wErrors {text_wErrors}")
    text_corrected = proofread(text_wErrors)
    # print(f"text_corrected {text_corrected}")
    
    # Compare texts without commas and dots for isError
    text_wErrors_no_commas_dots = remove_commas_and_dots(text_wErrors)
    # print(f"text_wErrors_no_commas_dots {text_wErrors_no_commas_dots}")
    text_corrected_no_commas_dots = remove_commas_and_dots(text_corrected)
    # print(f"text_corrected_no_commas_dots {text_corrected_no_commas_dots}")
    
    isWordCountSame = np.abs(count_words(text_wErrors_no_commas_dots) - count_words(text_corrected_no_commas_dots)) < 2
    isError =  isWordCountSame and (text_corrected_no_commas_dots.lower() != text_wErrors_no_commas_dots.lower())
    # print(f"isError {isError}")
    text_corrected = remove_space_before_punctuation(text_corrected)

    # Tokenize the texts, treating hyphenated words as single tokens
    tokens1 = re.findall(r'\w+(?:-\w+)*\.?|\.', text_wErrors)
    tokens2 = re.findall(r'\w+(?:-\w+)*\.?|\.', text_corrected)

    # Create a Differ object
    d = difflib.Differ()

    # Compare the tokens
    diff = list(d.compare(tokens1, tokens2))

    # Reconstruct the texts with markers
    text_with_removed_markers = []
    text_with_added_markers = []
    for token in diff:
        if token.startswith('- '):
            text_with_removed_markers.append('-' + token[2:] + '-')
        elif token.startswith('+ '):
            text_with_added_markers.append('<' + token[2:] + '>')
        elif token.startswith('  '):
            text_with_removed_markers.append(token[2:])
            text_with_added_markers.append(token[2:])

    reconstructed_original = ' '.join(text_with_removed_markers)
    reconstructed_corrected = ' '.join(text_with_added_markers)
    return bool(isError), reconstructed_original, reconstructed_corrected

In [8]:
grammarly('ich liebe dich')


(False, '-ich- liebe dich', '<Ich> liebe dich')